In [1]:
#import sparql_dataframe
from pandas import *
from queryWikidata import query_wikidata

In [2]:
endpoint = "https://query.wikidata.org/sparql"
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:105.0) Gecko/20100101 Firefox/105.0"

In [ ]:
""" {
    ?historicalCountry wdt:P706 wd:Q145694 . #historicalCountry is anything located in the Italian peninsula
    } UNION {
      ?historicalCountry wdt:P17 wd:Q38 .     #and anything of the sovereign state of Italy
    }
    ?writer (wdt:P27 | p:P27 / ps:P27) ?historicalCountry . #get writers having or having had the citizenship of one of those countries
  } UNION {
    VALUES ?mcItaly {wd:Q38 wd:Q172579 wd:Q223936 wd:Q48742118} #modern/contemporary Italy can be one of these entities
    ?writer (wdt:P27 | p:P27 / ps:P27) ?mcItaly . #get writers having or having had the citizenship of one of those countries
  } UNION { """

In [29]:
wq_post_1200 = """
SELECT DISTINCT ?writer ?writerLabel ?imdb
WHERE {
  
    VALUES ?mcItaly {wd:Q38 wd:Q172579 wd:Q223936 wd:Q48742118} #modern/contemporary Italy can be one of these entities
    ?writer wdt:P31 wd:Q5 ;
            wdt:P27 ?country ;
            wdt:P345 ?imdb .
          # wdt:P569 ?dob .                          #the block gets the writers having had the citizenship of a country, whose capital city is now an Italian city
    ?country (wdt:P36 | p:P36 / ps:P36) ?capitalCity .  #However, it seems to provide no addition to the results
    ?capitalCity wdt:P31 wd:Q515 ; #get cities
                  wdt:P17 ?mcItaly .

  
  {
    VALUES ?otherRole {wd:Q6625963 wd:Q49757}   #get writers that, if screenwriters, are also either novelists or poets
    ?writer wdt:P106 wd:Q28389 ;
           wdt:P106 ?otherRole.
  }UNION{
    VALUES ?role {wd:Q6625963 wd:Q49757 wd:Q36180 wd:Q12144794}  #get writers that are novelists, poets, prosaists, common wikidata "writers" but NOT screenwriters
    ?writer (wdt:P106 | p:P106 / ps:P106) ?role .
    FILTER NOT EXISTS {?writer wdt:P106 wd:Q28389}
  }
  
  # FILTER ("1200-01-01"^^xsd:dateTime <= ?dob) 
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}

GROUP BY ?writer ?writerLabel ?imdb
"""

writers_post_1200_df = query_wikidata(endpoint, wq_post_1200, user_agent, True).fillna("")
writers_post_1200_df

EndPointInternalError: EndPointInternalError: endpoint returned code 500 and response. 

Response:
b'SPARQL-QUERY: queryStr=\nSELECT DISTINCT ?writer ?writerLabel ?imdb\nWHERE {\n  \n    VALUES ?mcItaly {wd:Q38 wd:Q172579 wd:Q223936 wd:Q48742118} #modern/contemporary Italy can be one of these entities\n    ?writer wdt:P31 wd:Q5 ;\n            wdt:P27 ?country  .\n          # wdt:P569 ?dob .                          #the block gets the writers having had the citizenship of a country, whose capital city is now an Italian city\n    OPTIONAL {?writer wdt:P345 ?imdb}\n    ?country (wdt:P36 | p:P36 / ps:P36) ?capitalCity .  #However, it seems to provide no addition to the results\n    ?capitalCity wdt:P31 wd:Q515 ; #get cities\n                  wdt:P17 ?mcItaly .\n\n    \n\n\n  {\n    VALUES ?otherRole {wd:Q6625963 wd:Q49757}   #get writers that, if screenwriters, are also either novelists or poets\n    ?writer wdt:P106 wd:Q28389 ;\n           wdt:P106 ?otherRole.\n  }UNION{\n    VALUES ?ro

In [30]:
qf8 = """
SELECT DISTINCT ?person ?personLabel ?authorImdbId
(COUNT(DISTINCT ?work) AS ?work) (COUNT(DISTINCT ?derivativeWork) AS ?derivativeWork)
(COUNT(DISTINCT ?workCreated) AS ?creatorP) (COUNT(DISTINCT ?workAuthored) AS ?authorP) (COUNT(DISTINCT ?workInAuthorsLists) AS ?workInAuthorsLists)
(COUNT(DISTINCT ?occupationWriter) AS ?writer) (COUNT(DISTINCT ?occupationNovelist) AS ?novelist) (COUNT(DISTINCT ?occupationPoet) AS ?poet) (COUNT(DISTINCT ?occupationProsaist) AS ?prosaist) (COUNT(DISTINCT ?occupationScreenwriter) AS ?screenwriter)
(COUNT(DISTINCT ?influencedWork) AS ?influencedByP) (COUNT(DISTINCT ?workBasedOn) AS ?basedOnP) (COUNT(DISTINCT ?referencedWork) AS ?referencesWorkP) (COUNT(DISTINCT ?derivedWork) AS ?derivedWorkP) (COUNT(DISTINCT ?inspiredWork) AS ?inspiredByP)  (COUNT(DISTINCT ?workAfter) AS ?afterWorkByP)

WHERE {

      VALUES ?mcItaly {wd:Q38 wd:Q172579 wd:Q223936 wd:Q48742118} #modern/contemporary Italy can be one of these entities
    ?person wdt:P31 wd:Q5 ;
            wdt:P27 ?country .                          #the block gets the writers having had the citizenship of a country, whose capital city is now an Italian city
    ?country (wdt:P36 | p:P36 / ps:P36) ?capitalCity .  #However, it seems to provide no addition to the results
    ?capitalCity wdt:P31 wd:Q515 ; #get cities
                  wdt:P17 ?mcItaly .
                  
    {
      ?person ?occupationWriter  wd:Q36180.
    }UNION{
      ?person ?occupationNovelist  wd:Q6625963
    }UNION{
      ?person ?occupationPoet  wd:Q49757
    }UNION{
      ?person ?occupationProsaist  wd:Q12144794
    }UNION{
      ?person ?occupationScreenwriter wd:Q28389
    }
    


  

  

    {
      ?work wdt:P170 ?person .
      BIND (?work AS ?workCreated)
     }UNION{
      ?work wdt:P50 ?person .                             #attenzione: mi connette ad ogni opera, quindi se uno è segnato come writer ma è anchre pittore avrò anche le sue opere da pittore
      BIND (?work AS ?workAuthored)
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?person ?authorOf ?work .
      BIND (?work AS ?workInAuthorsLists)
    }

  
  {
    VALUES ?writtenWork {wd:Q7725634 wd:Q47461344 wd:Q571 wd:Q3331189}  #here I specify the kind of work we are interested in
    ?work wdt:P31 ?writtenWork .
  }UNION{
    ?derivativeWork wdt:P737 ?work . 
    BIND(?derivativeWork AS ?influencedWork)
  }UNION{
    ?derivativeWork wdt:P144 ?work .
    BIND(?derivativeWork AS ?workBasedOn)
  }UNION{
    ?derivativeWork wdt:P941 ?work .
    BIND(?derivativeWork AS ?inspiredWork)
  }UNION{  
    ?derivativeWork wdt:P8371 ?work .
    BIND(?derivativeWork AS ?referencedWork)
  }UNION{
    ?work wdt:P4969 ?derivativeWork .
    BIND(?derivativeWork AS ?derivedWork)
  }UNION{
    ?derivativeWork wdt:P1877 ?person .
    BIND(?derivativeWork AS ?workAfter)
  }
  
  
  OPTIONAL {?person wdt:P345 ?authorImdbId}
 
                              
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
GROUP BY ?person ?personLabel ?authorImdbId
"""

df8 = query_wikidata(endpoint, qf8, user_agent, True).fillna("")
df8

EndPointInternalError: EndPointInternalError: endpoint returned code 500 and response. 

Response:
b'SPARQL-QUERY: queryStr=\nSELECT DISTINCT ?person ?personLabel ?authorImdbId\n(COUNT(DISTINCT ?work) AS ?work) (COUNT(DISTINCT ?derivativeWork) AS ?derivativeWork)\n(COUNT(DISTINCT ?workCreated) AS ?creatorP) (COUNT(DISTINCT ?workAuthored) AS ?authorP) (COUNT(DISTINCT ?workInAuthorsLists) AS ?workInAuthorsLists)\n(COUNT(DISTINCT ?occupationWriter) AS ?writer) (COUNT(DISTINCT ?occupationNovelist) AS ?novelist) (COUNT(DISTINCT ?occupationPoet) AS ?poet) (COUNT(DISTINCT ?occupationProsaist) AS ?prosaist) (COUNT(DISTINCT ?occupationScreenwriter) AS ?screenwriter)\n(COUNT(DISTINCT ?influencedWork) AS ?influencedByP) (COUNT(DISTINCT ?workBasedOn) AS ?basedOnP) (COUNT(DISTINCT ?referencedWork) AS ?referencesWorkP) (COUNT(DISTINCT ?derivedWork) AS ?derivedWorkP) (COUNT(DISTINCT ?inspiredWork) AS ?inspiredByP)  (COUNT(DISTINCT ?workAfter) AS ?afterWorkByP)\n\nWHERE {\n\n      VALUES ?mcItaly {wd: